# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre and '2024' in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    937 non-null    int64  
 1   categoria             937 non-null    object 
 2   propiedad             937 non-null    object 
 3   precio                937 non-null    int64  
 4   metros_total          937 non-null    int64  
 5   metros_construido     937 non-null    int64  
 6   precio_m2_construido  937 non-null    float64
 7   estacionamiento       937 non-null    int64  
 8   recamaras             937 non-null    int64  
 9   baño                  937 non-null    int64  
 10  medio_baño            937 non-null    int64  
 11  baño_total            937 non-null    float64
dtypes: float64(2), int64(8), object(2)
memory usage: 88.0+ KB


In [6]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None
df_total = df_total.drop(columns=['categoria'], errors='ignore')


# Convertir 'precio' a numérico y manejar errores para tener la categoría correcta 
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]


df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [7]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=10) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 


df_total = df_total[df_total['precio'] != 0]
#df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [8]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        6.0    6.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        6.0    6.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Casa Mediana

In [9]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    1.0
B                      2.0        3.0    2.0
C                      2.0        3.0    1.0
D                      2.0        3.0    1.0
E                      2.0        3.0    1.0
L                      2.0        6.0    6.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      1  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    1.0
B                      2.0        3.0    2.0
C                      2.0        3.0    1.0
D                      2.0        3.0    1.0
E                      2.0        3.0    1.0
L                      2.0        6.0    6.0
S                      2.0        3.0    2.0
general                2.0        3.0    1.0


# Precio Mercado

In [10]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3300000.0,3946465.0,3.582541e+06,3555000.0
1,B,2600000.0,3200000.0,2.883613e+06,2890000.0
2,C,1800000.0,2499000.0,2.131160e+06,2159000.0
3,D,1020000.0,1737200.0,1.278118e+06,1266000.0
4,E,346040.0,995000.0,6.505581e+05,600000.0
5,L,17000000.0,17500000.0,1.725000e+07,17250000.0
6,S,750000.0,10000000.0,3.258176e+06,3000000.0


In [11]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,300,000.00
1     $2,600,000.00
2     $1,800,000.00
3     $1,020,000.00
4       $346,040.00
5    $17,000,000.00
6       $750,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,946,465.00
1     $3,200,000.00
2     $2,499,000.00
3     $1,737,200.00
4       $995,000.00
5    $17,500,000.00
6    $10,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descrip

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95","$3,555,000.00"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90","$2,890,000.00"
2,C,"$1,800,000.00","$2,499,000.00","$2,131,159.98","$2,159,000.00"
3,D,"$1,020,000.00","$1,737,200.00","$1,278,118.23","$1,266,000.00"
4,E,"$346,040.00","$995,000.00","$650,558.06","$600,000.00"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00","$17,250,000.00"
6,S,"$750,000.00","$10,000,000.00","$3,258,176.47","$3,000,000.00"


In [12]:
todo_descripcion = df_total[(~df_total['metros_construido'].isin([0,1,2,3,4,5]))]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\4176233628.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,023.81
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\4176233628.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $34,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\4176233628.py:13: FutureWarning: Setting an item of incompatible dtype is d

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,"$1,023.81","$34,000.00","$8,948.62","$8,158.33",general


In [13]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [14]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95","$3,555,000.00"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90","$2,890,000.00"
2,C,"$1,800,000.00","$2,499,000.00","$2,131,159.98","$2,159,000.00"
3,D,"$1,020,000.00","$1,737,200.00","$1,278,118.23","$1,266,000.00"
4,E,"$346,040.00","$995,000.00","$650,558.06","$600,000.00"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00","$17,250,000.00"
6,S,"$750,000.00","$10,000,000.00","$3,258,176.47","$3,000,000.00"
7,general,"$1,023.81","$34,000.00","$8,948.62","$8,158.33"


In [15]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [16]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [17]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,E,"$346,040.00","$995,000.00","$650,558.06","$600,000.00"
1,D,"$1,020,000.00","$1,737,200.00","$1,278,118.23","$1,266,000.00"
2,C,"$1,800,000.00","$2,499,000.00","$2,131,159.98","$2,159,000.00"
3,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90","$2,890,000.00"
4,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95","$3,555,000.00"
5,S,"$3,300,000.00","$10,000,000.00","$3,582,540.95","$3,555,000.00"
6,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00","$17,250,000.00"
7,ELITE,$nan,$nan,$nan,$nan
8,general,"$1,023.81","$34,000.00","$8,948.62","$8,158.33"


# M2 Terreno

In [18]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\1389371296.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      150.00 m²
1       70.00 m²
2       80.00 m²
3       70.00 m²
4       34.00 m²
5    1,000.00 m²
6       82.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\1389371296.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1,049.00 m²
1      232.00 m²
2      400.00 m²
3      400.00 m²
4      398.00 m²
5    1,972.00 m²
6    2,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,"1,049.00 m²",297.19 m²,200.00 m²
1,B,70.00 m²,232.00 m²,164.65 m²,200.00 m²
2,C,80.00 m²,400.00 m²,172.00 m²,120.00 m²
3,D,70.00 m²,400.00 m²,137.77 m²,114.00 m²
4,E,34.00 m²,398.00 m²,139.05 m²,119.00 m²
5,L,"1,000.00 m²","1,972.00 m²","1,486.00 m²","1,486.00 m²"
6,S,82.00 m²,"2,500.00 m²",317.13 m²,200.00 m²


In [19]:
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1)&(df_total_terreno['metros_total'] > 20)]


todo_descripcion = df_total[df_total['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\589930878.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\589930878.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    2,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\589930878.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,34.00 m²,"2,500.00 m²",175.00 m²,125.00 m²,general


In [20]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [21]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,"1,049.00 m²",297.19 m²,200.00 m²
1,B,70.00 m²,232.00 m²,164.65 m²,200.00 m²
2,C,80.00 m²,400.00 m²,172.00 m²,120.00 m²
3,D,70.00 m²,400.00 m²,137.77 m²,114.00 m²
4,E,34.00 m²,398.00 m²,139.05 m²,119.00 m²
5,L,"1,000.00 m²","1,972.00 m²","1,486.00 m²","1,486.00 m²"
6,S,82.00 m²,"2,500.00 m²",317.13 m²,200.00 m²
7,general,34.00 m²,"2,500.00 m²",175.00 m²,125.00 m²


# M2 Construcción

In [22]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\3111444512.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    150.00 m²
1     95.00 m²
2     90.00 m²
3     62.00 m²
4     41.00 m²
5    500.00 m²
6     81.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\3111444512.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    400.00 m²
1    600.00 m²
2    404.00 m²
3    420.00 m²
4    420.00 m²
5    520.00 m²
6    460.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,400.00 m²,253.67 m²,200.00 m²
1,B,95.00 m²,600.00 m²,204.18 m²,150.00 m²
2,C,90.00 m²,404.00 m²,202.16 m²,159.00 m²
3,D,62.00 m²,420.00 m²,139.65 m²,120.00 m²
4,E,41.00 m²,420.00 m²,173.73 m²,178.00 m²
5,L,500.00 m²,520.00 m²,510.00 m²,510.00 m²
6,S,81.00 m²,460.00 m²,247.65 m²,240.00 m²


In [23]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    2,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,34.00 m²,"2,500.00 m²",175.00 m²,125.00 m²,general


In [24]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [25]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,400.00 m²,253.67 m²,200.00 m²
1,B,95.00 m²,600.00 m²,204.18 m²,150.00 m²
2,C,90.00 m²,404.00 m²,202.16 m²,159.00 m²
3,D,62.00 m²,420.00 m²,139.65 m²,120.00 m²
4,E,41.00 m²,420.00 m²,173.73 m²,178.00 m²
5,L,500.00 m²,520.00 m²,510.00 m²,510.00 m²
6,S,81.00 m²,460.00 m²,247.65 m²,240.00 m²
7,general,34.00 m²,"2,500.00 m²",175.00 m²,125.00 m²


# Pecio M2 Construcción

In [26]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3]) & (df_total['precio_m2_construido'] > 1000)]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\2458575394.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,912.50
1     $5,333.33
2     $4,918.81
3     $2,607.14
4     $1,023.81
5    $33,653.85
6     $2,777.78
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\2458575394.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $26,309.77
1    $30,000.00
2    $21,000.00
3    $20,879.67
4    $15,806.45
5    $34,000.00
6    $23,809.52
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).app

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,912.50","$26,309.77","$15,535.85","$16,500.00"
1,B,"$5,333.33","$30,000.00","$17,379.56","$19,082.25"
2,C,"$4,918.81","$21,000.00","$12,082.48","$13,402.78"
3,D,"$2,607.14","$20,879.67","$11,328.20","$11,328.13"
4,E,"$1,023.81","$15,806.45","$5,489.51","$3,834.95"
5,L,"$33,653.85","$34,000.00","$33,826.93","$33,826.93"
6,S,"$2,777.78","$23,809.52","$12,937.75","$13,361.11"


In [27]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,023.81
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $34,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_13916\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is depr

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,"$1,023.81","$34,000.00","$8,948.62","$8,158.33",general


In [28]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [29]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,912.50","$26,309.77","$15,535.85","$16,500.00"
1,B,"$5,333.33","$30,000.00","$17,379.56","$19,082.25"
2,C,"$4,918.81","$21,000.00","$12,082.48","$13,402.78"
3,D,"$2,607.14","$20,879.67","$11,328.20","$11,328.13"
4,E,"$1,023.81","$15,806.45","$5,489.51","$3,834.95"
5,L,"$33,653.85","$34,000.00","$33,826.93","$33,826.93"
6,S,"$2,777.78","$23,809.52","$12,937.75","$13,361.11"
7,general,"$1,023.81","$34,000.00","$8,948.62","$8,158.33"
